In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

# Part 1

# Data Scraping

In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [3]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [4]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"Postal Code": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [5]:
#drop cells with Borough not assigned
toronto_dropped = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_dropped.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
# group multiple neighborhoods having same postal code
toronto_group = toronto_dropped.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
toronto_group['Neighborhood'] = toronto_group['Neighborhood'].str.replace('/', ',')
toronto_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# rows of dataframe
toronto_group.shape

(103, 3)

# Part 2

# Adding Lattitude and Longitude to the dataframe

In [8]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_id = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='enter ibm_api_key_id',
    ibm_auth_endpoint="enter ibm_auth_endpoint",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_id.get_object(Bucket='bucket',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )


In [9]:
coordinates = pd.read_csv(body)
coordinates.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merge the two dataframes

In [10]:
dataframe = pd.merge(toronto_group, coordinates, on='Postal Code')
dataframe.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### save these into csv files

In [11]:
toronto_group.to_csv('toronto_postal_codes.csv', index = False)
dataframe.to_csv('final_toronto_geospatial.csv', index = False)

# Part 3

In [12]:
print('The dataframe has {} boroughs'.format(
        len(dataframe['Borough'].unique())
    )
)

The dataframe has 10 boroughs


#### Use geopy library to get the latitude and longitude values of Toronto City

In [15]:
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

address = 'Toronto , Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dataframe['Latitude'], dataframe['Longitude'], dataframe['Borough'], dataframe['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### We segment and cluster only the neighborhoods in Scarborough

In [17]:
scarborough_data = dataframe[dataframe['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [19]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Borough'], scarborough_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

#### Define Foursquare Credentials and Version

In [ ]:
# @hidden_cell

CLIENT_ID = 'enter foursquare client id' # your Foursquare ID
CLIENT_SECRET = 'enter foursquare client secret' # your Foursquare Secret
VERSION = 'enter foursquare api version' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe

In [21]:
scarborough_data.loc[0, 'Neighborhood']

'Malvern , Rouge'

#### Get the neighborhood's latitude and longitude values

In [22]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern , Rouge are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Malvern , Rouge within a radius of 500 meters

In [23]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9c70dac546f3001be9edd5'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Clean the json and structure it into a pandas dataframe

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


#### Function to repeat the same process to all the neighborhoods in Scarborough

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge


In [29]:
print(scarborough_venues.shape)
scarborough_venues.head()

(96, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


 #### No. of venues were returned for each neighborhood

In [30]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners , Tam O'Shanter , Sullivan",14,14,14,14,14,14
"Cliffside , Cliffcrest , Scarborough Village West",2,2,2,2,2,2
"Dorset Park , Wexford Heights , Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile , Clairlea , Oakridge",9,9,9,9,9,9
"Guildwood , Morningside , West Hill",7,7,7,7,7,7
"Kennedy Park , Ionview , East Birchmount Park",8,8,8,8,8,8


#### Unique categories curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 58 uniques categories.


## Analyze Each Neighborhood

In [32]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant,Women's Store
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
scarborough_onehot.shape

(96, 59)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant,Women's Store
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
1,"Birch Cliff , Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.125000,0.000000,0.000,0.000000,0.000000
3,"Clarks Corners , Tam O'Shanter , Sullivan",0.0,0.000,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,...,0.000000,0.071429,0.00,0.000000,0.000000,0.071429,0.000000,0.000,0.000000,0.000000
4,"Cliffside , Cliffcrest , Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
5,"Dorset Park , Wexford Heights , Scarborough To...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000,0.166667,0.000000
6,"Golden Mile , Clairlea , Oakridge",0.0,0.000,0.000000,0.222222,0.000000,0.0,0.000000,0.000000,0.222222,...,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
7,"Guildwood , Morningside , West Hill",0.0,0.000,0.000000,0.000000,0.142857,0.0,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
8,"Kennedy Park , Ionview , East Birchmount Park",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000
9,"Malvern , Rouge",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000


In [35]:
scarborough_grouped.shape

(16, 59)

#### Print each neighborhood along with the top 5 most common venues

In [36]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Birch Cliff , Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1           Skating Rink  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.12
1  Caribbean Restaurant  0.12
2                Bakery  0.12
3                  Bank  0.12
4    Athletics & Sports  0.12


----Clarks Corners , Tam O'Shanter , Sullivan----
                 venue  freq
0          Pizza Place  0.14
1    Convenience Store  0.07
2   Chinese Restaurant  0.07
3             Pharmacy  0.07
4  Fried Chicken Joint  0.07


----Cliffside , Cliffcrest , Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1  

#### Put this into a *pandas* dataframe

In [37]:
#function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
#create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Women's Store,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff , Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Women's Store,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Hakka Restaurant,Bakery,Caribbean Restaurant,Fried Chicken Joint,Bank,Gas Station,Thai Restaurant,Athletics & Sports,General Entertainment,Construction & Landscaping
3,"Clarks Corners , Tam O'Shanter , Sullivan",Pizza Place,Noodle House,Shopping Mall,Convenience Store,Pharmacy,Coffee Shop,Italian Restaurant,Bank,Fast Food Restaurant,Thai Restaurant
4,"Cliffside , Cliffcrest , Scarborough Village West",American Restaurant,Motel,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


## Cluster Neighborhoods

In [39]:
#Run k-means to cluster the neighborhood into 5 clusters

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 1, 1, 2], dtype=int32)

In [40]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Women's Store,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,0.0,Bar,Construction & Landscaping,Women's Store,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,1.0,Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Electronics Store,Rental Car Location,Women's Store,Discount Store,Cosmetics Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Bakery,Caribbean Restaurant,Fried Chicken Joint,Bank,Gas Station,Thai Restaurant,Athletics & Sports,General Entertainment,Construction & Landscaping


In [41]:
#visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [42]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Bar,Construction & Landscaping,Women's Store,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


#### Cluster 2

In [43]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Mexican Restaurant,Intersection,Bank,Medical Center,Breakfast Spot,Electronics Store,Rental Car Location,Women's Store,Discount Store,Cosmetics Shop
4,Scarborough,1.0,Hakka Restaurant,Bakery,Caribbean Restaurant,Fried Chicken Joint,Bank,Gas Station,Thai Restaurant,Athletics & Sports,General Entertainment,Construction & Landscaping
5,Scarborough,1.0,Women's Store,Jewelry Store,Playground,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,Scarborough,1.0,Discount Store,Chinese Restaurant,Train Station,Department Store,Bus Station,Hobby Shop,Coffee Shop,Construction & Landscaping,Grocery Store,General Entertainment
7,Scarborough,1.0,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Intersection,Metro Station,Park,Bar,Breakfast Spot,General Entertainment
9,Scarborough,1.0,College Stadium,General Entertainment,Skating Rink,Café,Women's Store,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1.0,Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Pet Store,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
11,Scarborough,1.0,Sandwich Place,Vietnamese Restaurant,Auto Garage,Bakery,Breakfast Spot,Shopping Mall,Middle Eastern Restaurant,Electronics Store,Convenience Store,Cosmetics Shop
12,Scarborough,1.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Women's Store,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1.0,Pizza Place,Noodle House,Shopping Mall,Convenience Store,Pharmacy,Coffee Shop,Italian Restaurant,Bank,Fast Food Restaurant,Thai Restaurant


#### Cluster 3

In [44]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Women's Store,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


#### Cluster 4

In [45]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3.0,Coffee Shop,Korean Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
14,Scarborough,3.0,Park,Coffee Shop,Playground,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 5

In [46]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4.0,American Restaurant,Motel,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
